The goal of this code is to scrape the data and put it in a file. Docker is needed to run this code. Part of this code is retrieved from another repository of me: https://github.com/janvanderdoe/amazon_nfu

# Import packages and start docker

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv
from collections import Counter
import docker
import re
import pandas as pd
import random

In [ ]:
client = docker.from_env()
container = client.containers.get('splash')
#start container
container.start()

In [ ]:
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'

In [ ]:
def get_soup(url):
    t = 0
    if url.startswith("https://www.amazon.com/dp/") or url.startswith("https://www.amazon.com/s?i"):
        time_wait = 1
    else:
        time_wait = 20
    while True:
        r = requests.get("http://localhost:8050/render.html", params={"url": url,'wait': time_wait, 'viewport': '320x480', 'images': 0, 'resource_timeout': 10}, headers={'User-Agent': user_agent})
        soup = BeautifulSoup(r.text, "html.parser")

        #check if h4 is nonetype
        if soup.find('h4') == None:
            pass
        elif soup.find('h4').text == 'Enter the characters you see below':
            raise Exception('Captcha')
        #print(soup.title.text)
        if soup.text.find('GlobalTimeoutError') != -1:
            container.restart()
            time.sleep(31)
            print(f"time sleep try {t}")
            time_wait += 4
            if t == 2:
                print(f"too many timeouts (2) for url: {url}")
                break
            t += 1
        else:
            print(f"page successfully loaded in time {time_wait}: {url}")
            break       
    return soup

# Product url scraping

In [ ]:
def get_link_asin(link):
    link_asin = {}
    splitted_link = link['href'].split('/')
    name = splitted_link[1]
    link_asin['link'] = name
    asin = splitted_link[3]
    link_asin['asin'] = asin
    #You can create the link to the review page with the name and asin
    link_asin['full_link'] = "https://www.amazon.com/"+str(name)+"/product-reviews/"+str(asin)+"/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber="
    return link_asin

In [ ]:
#with open(f"../../gen/input/amazon_links_per_category.csv", "w", encoding = "UTF-8", newline = '') as csv_file:
    writer = csv.writer(csv_file, delimiter = ";")
    writer.writerow(["name", "asin", "product_link", "page_link", "category"])


In [ ]:
all_product_links = []
category = "computer adapter"
i = 12
with open(f"../../gen/input/amazon_links_per_category.csv", "a", encoding = "UTF-8", newline = '') as csv_file:
    writer = csv.writer(csv_file, delimiter = ";")
    while i < 24:
        page_link = f"https://www.amazon.com/s?k=computer+adapter&i=electronics&rh=n%3A172282%2Cn%3A281407%2Cp_n_condition-type%3A2224371011&dc&page={i}&crid=1QAMSMU53HDFW&qid=1713623177&rnid=172282&sprefix=computer+adapter%2Caps%2C215&ref=sr_pg_{i}"
        soup = get_soup(page_link)
        print(f"currently looking at page {i}")
        #Block usually after about 100 pages
        if soup.title.text=="Sorry! Something went wrong!":
            print("Blocked")
            break
        #If not block, gather asin and link
        else:
            links = soup.find_all(class_="a-link-normal s-no-outline")
        for link in links:
            if link['href'].find('/dp/') != -1 and not link['href'].startswith('/dp/'):
                link_asin = get_link_asin(link)
                link_asin['page_link'] = page_link
                link_asin['category'] = category
                all_product_links.append(link_asin)
                writer.writerow(list(link_asin.values()))
        #Try to find next page button. If absent, break
        try:
            soup.find('span', class_ = "s-pagination-item s-pagination-next s-pagination-disabled").text
            print('last page')
            break
        except:
            pass
        print(len(all_product_links))
        i += 1
    #stop docker
    container.stop()
    container.start()

# Review scraping

In [ ]:
#with open(f"../../data/amazon_search_hedonic.csv", "w", encoding = "UTF-8", newline = '') as csv_file:
    writer = csv.writer(csv_file, delimiter = ";")
    writer.writerow(["asin_url",  "link", "product_title", "rating", "review_title", "country", "date", "review", "helpful", "picture",
            "verified", "video", "scrape_date"])

In [ ]:
#This function retrieves all the review info from a page
def get_reviews_from_page(product_info, start=1, end = 500):
    print(f"succesfully loaded {product_info['title']}")
    with open(f"../../data/amazon_search_hedonic.csv", "a", encoding = "UTF-8", newline = '') as csv_file:
        writer = csv.writer(csv_file, delimiter = ";")
        for i in range(start,end):
            missing_page = []
            soup_reviews = get_soup((product_info['link']+str(i)).replace("ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=", f"ref=cm_cr_getr_d_paging_btm_next_{i}?pageNumber="))
            
            #check if chaptcha
            if soup_reviews.find('h4') == None:
                pass
            elif soup_reviews.find('h4').text == 'Enter the characters you see below':
                print('chaptcha check')
                #container.stop()
                raise Exception('chaptcha')
            reviews = soup_reviews.find_all('div', {"data-hook":"review"})
            if len(reviews) == 0:
                print(f'Could not find any reviews at page {i}')
                break
            else:
                pass



            for review in reviews:
                review_info = {}
                review_info = {
                'product_url' : product_info['link']+str(i),
                'link' : product_info['link']+str(i),
                'product_title' : product_info['title']
                }
                try:
                    review_info['star_rating'] = float(review.find('i', {"data-hook":"review-star-rating"}).text.replace(' out of 5 stars',''))
                except:
                    #foreign countries
                    try:
                        review_info['star_rating'] = float(review.find('i', {"data-hook":"cmps-review-star-rating"}).text.replace(' out of 5 stars',''))
                    except:
                        review_info['star_rating'] = None
                try:
                    review_info['title'] = review.find('a', {"data-hook":"review-title"}).text.strip()
                except:
                    try:
                        #foreign countries
                        review_info['title'] = review.find('span',{"data-hook":"review-title"}).text.strip()
                    except:
                        review_info['title'] = None
                try:
                    review_info['country'] = review.find('span', {"data-hook":"review-date"}).text.replace('Reviewed in ','').rpartition(' on')[0]
                except:
                    review_info['country'] = None
                try:
                    review_info['date'] = review.find('span', {"data-hook":"review-date"}).text.replace('Reviewed in ','').rpartition(' on')[2].strip()
                except:
                    review_info['date'] = None
                try:
                    span_elements = review.find('span', {"data-hook":"review-body"}).find_all('span')
                    filtered_span_elements = [span for span in span_elements if span.get('class') is None]
                    concatenated_text = ''.join([span.text for span in filtered_span_elements])
                    review_info['review'] = concatenated_text
                except:
                    review_info['review'] = None
                try:
                    helpful = review.find('span', {"data-hook" : "helpful-vote-statement"}).text.strip()
                    if helpful == "One person found this helpful":
                        review_info['helpful'] = 1
                    elif "found this helpful" in helpful:
                        review_info['helpful'] = helpful.split()[0]
                    else:
                        review_info['helpful'] = helpful
                except:
                    review_info['helpful'] = None
                try:
                    #get links for consumer pictures
                    if review.find('div', class_="review-image-tile-section") is None:
                        review_info['picture'] = 'no'
                    else:
                        pics = review.find('div', class_ = "a-section a-spacing-top-mini cr-lightbox-image-thumbnails").find_all('img')
                        pics = [img['src'] for img in pics]
                        review_info['picture'] = pics
                        #saving pics
                        try:
                            for pic in pics:
                                hq_pic = pic.replace("_SY88.jpg", "_SL1600_.jpg")
                                r = requests.get(hq_pic)
                                with open(f"../../gen/output/consumer_photos_category/"+hq_pic.split('/')[-1], "wb") as f:
                                    f.write(r.content)
                                time.sleep(0.1)
                        except:
                            print("Error while downloading pictures")
                except:
                    review_info['picture'] = None

                try:
                    if review.find('span', {"data-hook":"avp-badge"}) is None:
                        review_info['verified'] = 'no'
                    else:
                        review_info['verified'] = 'yes'
                except:
                    review_info['verified'] = None   
                try:
                    review_info['video'] = review.find('input', class_="video-url").get('value')
                except:
                    review_info['video'] = None
                review_info['scrape_date'] = time.strftime("%Y-%m-%d %H:%M:%S")
                all_reviews.append(review_info)
                missing_values = [key for key,value in review_info.items() if value == None]
                missing_page += missing_values
                writer.writerow(list(review_info.values()))
            print(len(all_reviews))
            if len(all_reviews) % 500 == 0:
                print('restart container')
                container.restart()
                time.sleep(15)
            #missing values checker
            if len(missing_page) > 0:
                print(f"the following items were missing: {dict(Counter(missing_page))}")

            if not soup_reviews.find('li', class_ = "a-disabled a-last"):
                pass
            else:
                print(f"page {i} was the last page")
                break

In [ ]:
#load csv using pandas
import pandas as pd
import re
all_reviews = []
#no headers
links_data = pd.read_csv(f"../../gen/input/amazon_links_per_category.csv", delimiter = ";")
links_data

In [ ]:
links_data[links_data['category'] == 'computer adapter']['page_link'][1484]

In [ ]:
def split_join_name(name):
    return '-'.join(name.split('-')[0:3])

links_data['name_part'] = links_data['name'].apply(split_join_name)
#remove duplicates in name_part
print(len(links_data))
links_data = links_data.drop_duplicates(subset='name_part', keep='first')
print(len(links_data))

In [ ]:
data_so_far = pd.read_csv(f"../../data/amazon_search_hedonic.csv", delimiter = ";")
data_so_far['product_title_clean'] = data_so_far['product_title'].apply(split_join_name)
#only keep links_data rows which name_part is not in product_title_clean
links_data = links_data[~links_data['name_part'].isin(data_so_far['product_title_clean'])]
print(len(links_data))

In [ ]:
def remove_last_digit(string):
    return re.sub(r'\d+$', '', string)


data_so_far['asin_url_clean'] = data_so_far['asin_url'].apply(remove_last_digit)
#merge with links_data
links_data_full = pd.read_csv(f"../../gen/input/amazon_links_per_category.csv", delimiter = ";")
data_so_far = pd.merge(data_so_far, links_data_full, left_on = 'asin_url_clean', right_on = 'product_link', how = 'left')
print(data_so_far['category'].value_counts())
print("----------------------")
#remove duplicates for link, product_title, review_title, review, date, name_x
data_so_far = data_so_far.drop_duplicates(subset = ['review_title', 'review', 'date', 'name_x', 'profile_url'])

#remove obs not from the united states
data_so_far = data_so_far[data_so_far['country'] == 'the United States']
data_so_far = data_so_far[data_so_far['review'] != '']
#show number of observations per category_y
print(data_so_far['category'].value_counts())

In [ ]:
all_reviews = []

#filter for dvd_player
links_data_filtered = links_data[links_data['category'] == 'smartphone']
links_so_far = data_so_far['asin_url'].unique()
print(len(links_data_filtered))
#remove last digits in links_so_far
links_so_far = [re.sub(r'\d+$', '', link) for link in links_so_far]
links_left = [links for links in links_data_filtered['product_link'] if links not in links_so_far]
print(len(links_left))

In [ ]:
#set random seed
import random
random.seed(42)
#shuffle links
random.shuffle(links_left)


In [ ]:
l = 1
scrape_links = links_left[100:200]
for link in scrape_links:
    print(f"{l} / {len(scrape_links)}")
    product_info = {
    'product_url' : "https://www.amazon.com/dp/"+link.split('/')[-2],
    'brand' : None,
    'model' : None,
    'link' : link,
    'title' : link.split('/')[3],
    'product_title' : None
    }
    get_reviews_from_page(product_info, 1, 13)
    l += 1
    if l % 10 == 0:
        print('restart container')
        container.restart()
        time.sleep(10)
container.stop()

## consumer photos

In [ ]:
consumer_photos = data['picture'].unique()[1:]

In [ ]:
import ast
def get_links(pics):
    list_pics = ast.literal_eval(pics)
    return [pic.split('/')[-1] for pic in list_pics]

#apply to consumer)photos
consumer_photos_clean = [get_links(pics) for pics in consumer_photos]
consumer_photos_clean = [item for sublist in consumer_photos_clean for item in sublist]

In [ ]:
#get all existing consumer photos
consumer_photos = os.listdir("../../gen/output/consumer_photos_category")
cp_left = [cp for cp in consumer_photos_clean if cp.replace('_SY88.jpg', '_SL1600_.jpg') not in consumer_photos]
cp_left

In [ ]:
#scrape consumer photos
for cp in cp_left:
    try:
        cp = cp.replace('_SY88.jpg', '_SL1600_.jpg')
        r = requests.get("https://images-na.ssl-images-amazon.com/images/I/"+cp)
        with open(f"../../gen/output/consumer_photos_category/"+cp, "wb") as f:
            f.write(r.content)
        time.sleep(0.1)
    except:
        print(f"error with {cp}")